# Zero-Shot Model Evaluation

# 1. Initialize environment and model configuration

Import necessary libraries and modules from the torch and transformers packages. 

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

Define the path to the pre-trained model directory. Set up configurations for quantization, offering choices for both 4-bit and 8-bit quantization.

In [2]:
model_path = "../Model_pretrained/vicuna-7b-v1.5"

use_4bit = True
use_8bit = True
quant_type_4bit = "nf4"
quant_type_8bit = "nf8"
use_nested_quant = True
compute_dtype_bfloat16 = torch.bfloat16
compute_dtype_float16 = torch.float16
device_mapping = {"": 0}

""" Alternative parameters:
load_in_8bit_fp32_cpu_offload=True,  # Enable CPU offloading for certain layers
bnb_8bit_blocksparse_layout=None,  # Block-sparse layout, use None for dense models
bnb_8bit_custom_kernel=False,  # Use custom kernel, false by default
bnb_8bit_cpu_offload=True,  # Enable CPU offloading
bnb_8bit_cpu_offload_dtype=torch.float32,  # Data type for CPU offloaded tensors
bnb_8bit_cpu_offload_use_pin_memory=True,  # Use pinned memory for CPU offloading
bnb_8bit_cpu_offload_use_fast_fp32_to_fp16_conversion=False  # Use fast conversion from FP32 to FP16
"""

bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = quant_type_4bit,
    bnb_4bit_use_double_quant = use_nested_quant,
    bnb_4bit_compute_dtype = compute_dtype_bfloat16
)
bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit = use_8bit,
    bnb_8bit_quant_type = quant_type_8bit,
    bnb_8bit_use_double_quant = use_nested_quant,
    bnb_8bit_compute_dtype = compute_dtype_float16
)

# 2. Load pretrained model

Load the tokenizer and model by selecting one of the configurations provided (4-bit or 8-bit).

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config = bnb_config_4bit, device_map = device_mapping)

bin C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\generation\configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Print the loaded model for inspection or verification.

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSN

# 3. Text generation pipeline: prompting test

Set up a text generation pipeline using the loaded model and tokenizer. Define a prompt for text generation related to a financial question. Generate text sequences based on the provided prompt using the text generation pipeline. Print the generated text sequences.

In [4]:
text_generation_pipeline = pipeline('text-generation', model = model, tokenizer = tokenizer)

prompt = """
Question: Current ratio of a company is 2.10 in 2021, placing its value in the category defined as greater than 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?
Answer:
"""
sequences = text_generation_pipeline(
    prompt,
    max_length = 512,
    do_sample = True,
    top_k = 10,
    temperature = 0.6,
    return_full_text = False,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: In 2021, the company had a high Current ratio, indicating that it has a significant amount of current assets relative to its current liabilities. This may suggest that the company is likely to meet its short-term obligations without significant challenges. However, it also implies that the company may have a potential liquidity overload, as excessively high ratios may indicate that resources are being inefficiently utilized. In 2022, the Current ratio has decreased, moving from the high range to a range indicative of a balanced liquidity position, where the company is able to cover its short-term liabilities without significant challenges, and resources are being utilized efficiently.


# 4. Load sample: formatting and splitting by ratio type

Import the pandas library and set an option to display the maximum column width to None (unlimited width). Define the path, file name and file type (extension) for the CSV file to be read. Read the CSV file into a pandas DataFrame using the specified path and file details. Display the first few rows of the DataFrame.

In [5]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

path = '../Sample_generation/Sample/'
file = 'sample'
type_file = '.csv'

sample = pd.read_csv(path+file+type_file, index_col = 0)

sample.head()

,instruction,analysis,ratio,company
0,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.",Current,3M
1,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.",Quick,3M
2,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.",Cash,3M
3,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",Current,Adobe
4,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquid

Define a function named format_and_split_df that takes the DataFrame as input and performs the following operations: Reindex the DataFrame with columns 'company', 'ratio', 'instruction', and 'analysis'. Rename the 'analysis' column to 'analysis_expected'. Add a new column 'analysis_generated' with all values set to None. Group the DataFrame by the 'ratio' column. Create three separate DataFrames based on the 'ratio' values (Current, Quick, Cash). Return a list containing these three DataFrames.

In [6]:
def format_and_split_df(df):
    dataframe = df.reindex(['company', 'ratio', 'instruction', 'analysis'], axis=1)
    dataframe = dataframe.rename(columns={'analysis': 'analysis_expected'})
    dataframe['analysis_generated'] = None
    
    grouped_df = dataframe.groupby('ratio')
    
    current_ratio_df = grouped_df.get_group('Current').reset_index(drop=True)
    quick_ratio_df = grouped_df.get_group('Quick').reset_index(drop=True)
    cash_ratio_df = grouped_df.get_group('Cash').reset_index(drop=True)
    
    return [current_ratio_df, quick_ratio_df, cash_ratio_df]

Unpack the list of DataFrames into three separate variables: current_ratio_df, quick_ratio_df, and cash_ratio_df. Display the first few rows of the resulting DataFrames.

In [7]:
dataframes = format_and_split_df(sample)

current_ratio_df, quick_ratio_df, cash_ratio_df = dataframes

In [8]:
current_ratio_df.head()

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Current,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.",None
1,Adobe,Current,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",None
2,Airbnb,Current,"Current ratio of a company is 1.95 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events impacting liquidity. While the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.",None
3,Airbus,Current,"Current ratio of a company is 1.33 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.22, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the company possesses ample current assets to meet short-term debts, but there is a potential lack of readiness for unexpected events impacting liquidity. In 2022, the situation remains similar, but it is noteworthy that the ratio slightly decreases in value.",None
4,Amazon,Current,"Current ratio of a company is 1.14 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 0.94, placing its value in the category defined as less than 1.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, current assets (cash, accounts receivable, short-term investments and inventory) are enough to offset short-term debts. Nonetheless, the company might not be adequately prepared for unexpected events impacting its liquidity. In contrast, in 2022, ratio has decreased and is less than 1, signifying that current liabilities surpass current assets. This suggests a deficiency in liquidity, indicating that the company may struggle to meet its short-term debts.",None


# 5. Generate expected analysis

Import necessary modules from PyTorch for working with datasets and dataloaders. Define a custom dataset class named CustomDataset that inherits from Dataset. This class takes a DataFrame as input during initialization and implements methods for length (__len__) and getting individual items (__getitem__). It extracts the 'instruction' column from the DataFrame for each item.

Define a function generate_analysis that takes a DataFrame (dataframe) and a text generation pipeline (text_generation_pipeline) as input. Inside the function: Create a CustomDataset using the DataFrame passed as input of the function. Use a DataLoader to load the dataset with a batch size of 1. Iterate over the DataLoader batches, extracting the 'question' and generating a corresponding ratio analysis using the LLM. Store the generated answer in the 'analysis_generated' column of the DataFrame.

In [9]:
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        question = self.dataframe.iloc[idx]['instruction']
        return {
            'question': question
        }

def generate_analysis(dataframe, text_generation_pipeline):
    dataset = CustomDataset(dataframe)
    
    dataloader = DataLoader(dataset, batch_size=1)

    for idx, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
        question = batch['question'][0]
        
        prompt = f"Question: {question}\nAnswer:"
        
        sequences = text_generation_pipeline(
            prompt,
            max_length=512,
            do_sample=True,
            top_k=10,
            temperature = 0.6,
            return_full_text=False,
        )
        
        generated_text = sequences[0]['generated_text']
        dataframe.at[idx, 'analysis_generated'] = generated_text

    return dataframe

Set up a text generation pipeline using the LLM model and tokenizer.

In [10]:
text_generation_pipeline = pipeline('text-generation', model = model, tokenizer = tokenizer)

## 5.1 Expected analyses: Current ratio

Call the generate_analysis function using the Current ratio DataFrame created in step 4 and the text generation pipeline for updating the DataFrame with the analysis generated by the LLM. Display resulting DataFrame and save it to a CSV file.

In [15]:
current_ratio_df = generate_analysis(current_ratio_df, text_generation_pipeline)

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [16]:
current_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Current,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.","The current ratio of a company is a measure of its ability to pay its short-term liabilities with its short-term assets. A current ratio of 1.70 in 2021 indicates that the company had 1.70 times its current liabilities in current assets. This is considered a healthy ratio, as it shows that the company has enough resources to pay its short-term debts.\n\nIn 2022, the current ratio decreased to 1.54, which indicates that the company had 1.54 times its current liabilities in current assets. This is still considered a healthy ratio, but it shows a slight decrease in the company's ability to pay its short-term debts.\n\nOverall, the current ratio analysis for the years 2021 and 2022 shows that the company's ability to pay its short-term liabilities has remained stable, with a slight decrease in 2022."
1,Adobe,Current,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","The current ratio of a company is a measure of its ability to pay its short-term obligations as they come due. A current ratio of 1.25 in 2021 indicates that the company has $1.25 of current assets for every $1 of current liabilities. This suggests that the company has a relatively strong ability to pay its short-term obligations.\n\nIn 2022, the current ratio has decreased to 1.11, indicating that the company now has $1.11 of current assets for every $1 of current liabilities. This suggests that the company's ability to pay its short-term obligations has decreased slightly.\n\nIt is important to note that a current ratio of 1.00 or higher is generally considered to be a healthy sign, indicating that a company has enough current assets to cover its current liabilities. A ratio below 1.00 may indicate that a company is having difficulty paying its short-term obligations.\n\nTherefore, based on the current ratio analysis for the years 2021 and 2022, the company's ability to pay its short-term obligations has decreased slightly, but it is still within a healthy range."
2,Airbnb,Current,"Current ratio of a company is 1.95 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events impacting liquidity. While the ratio f

In [21]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_current_ratio'
type_file = '.csv'

current_ratio_df.to_csv(path+file+type_file, index=True)

## 5.2 Expected analyses: Quick ratio

Call the generate_analysis function using the Quick ratio DataFrame created in step 4 and the text generation pipeline for updating the DataFrame with the analysis generated by the LLM. Display resulting DataFrame and save it to a CSV file.

In [11]:
quick_ratio_df = generate_analysis(quick_ratio_df, text_generation_pipeline)

  0%|          | 0/50 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [12]:
quick_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Quick,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.","The quick ratio is a measure of a company's ability to meet its short-term obligations with its most liquid assets. A quick ratio of 1.08 in 2021 indicates that the company has a relatively high level of liquid assets compared to its short-term obligations, and is therefore in a strong position to meet its short-term obligations. A quick ratio of 0.93 in 2022, on the other hand, indicates that the company has a lower level of liquid assets compared to its short-term obligations, and may have a more difficult time meeting its short-term obligations.\n\nOverall, the quick ratio analysis for the years 2021 and 2022 shows a decrease in the company's ability to meet its short-term obligations with its most liquid assets. This suggests that the company may need to take steps to improve its liquidity position, such as increasing its liquid assets or reducing its short-term obligations."
1,Adobe,Quick,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquidity position has weakened compared to the previous year. While the quick ratio is still between 1.00 and 1.50, the decrease raises concerns about the ability to cover short-term liabilities with the most liquid assets.","The quick ratio is a measure of a company's ability to pay its short-term liabilities with its most liquid assets. It is calculated by dividing the sum of current assets (excluding inventory) by the sum of current liabilities. In 2021, the quick ratio was 1.11, indicating that the company had 1.11 times its current liabilities in current assets (excluding inventory) to cover its short-term debts. In 2022, the quick ratio decreased to 1.00, indicating that the company had only 1.00 times its current liabilities in current assets (excluding inventory) to cover its short-term debts. This suggests that the company's ability to pay its short-term liabilities has decreased."
2,Airbnb,Quick,"Quick ratio of a company is 1.89 in 2021, placing its value in the category defined as greater than 1.50. In 2022, Quick ratio has decreased to 1.81, placing its value in the category defined as greater than 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","In 2021 and 2022, current assets (excluding inventories and other less liquid assets) are more than sufficient to meet short-term curren

In [13]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_quick_ratio'
type_file = '.csv'

quick_ratio_df.to_csv(path+file+type_file, index=True)

## 5.3 Expected analyses: Cash ratio

Call the generate_analysis function using the Cash ratio DataFrame created in step 4 and the text generation pipeline for updating the DataFrame with the analysis generated by the LLM. Display resulting DataFrame and save it to a CSV file.

In [14]:
cash_ratio_df = generate_analysis(cash_ratio_df, text_generation_pipeline)

  0%|          | 0/50 [00:00<?, ?it/s]

C:\ProgramData\anaconda3\envs\tfg\Lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [15]:
cash_ratio_df

,company,ratio,instruction,analysis_expected,analysis_generated
0,3M,Cash,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.","Cash ratio analysis for the years 2021 and 2022 can be as follows:\n\nIn 2021, the cash ratio was 0.51, which is greater than 0.40, indicating that the company had a higher level of liquid assets than the industry average. This suggests that the company had a relatively strong financial position and was in a good position to meet its short-term obligations.\n\nIn 2022, the cash ratio decreased to 0.38, which is between 0.20 and 0.40. This indicates that the company's liquid assets as a percentage of its current liabilities decreased compared to the previous year. This could be a sign of weaker financial position, as the company may have had to rely more on external financing to meet its short-term obligations. However, it is important to consider other factors such as the company's operating cycle and the composition of its current liabilities when interpreting this ratio."
1,Adobe,Cash,"Cash ratio of a company is 0.55 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.52, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Both in 2021 and 2022, cash exceeds appropriate levels, so short-term resources are not being used adequately. Although the ratio has slightly decreased in 2022, the excess of liquidity persists.","The cash ratio analysis for the years 2021 and 2022 would indicate that the company's liquidity position has improved in 2022 compared to 2021. In 2021, the cash ratio was 0.55, which is higher than the value of 0.40 that defines a positive cash ratio. This suggests that the company had a relatively high level of liquidity in 2021. In 2022, the cash ratio decreased to 0.52, which is still greater than 0.40, indicating that the company's liquidity position has improved compared to the previous year. This suggests that the company has been able to generate more cash flow from its operations in 2022, which has helped to increase its liquidity position."
2,Airbnb,Cash,"Cash ratio of a company is 0.95 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.92, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the amount of cash surpasses 40% of short-term current liabilities. This suggests an opportunity to redirect resources for other purposes. It is important to note that, although the ratio has decreased in 2022, the excess liquidity situation has not been resolved.","The cash ratio is a liquidity ratio that measures a company's ability to pay its short-term liabilities with its short-term assets. A cash ratio greater than 1 indicates that a company has enough liquid assets to cover its short-term liabilities. In 2021, the cash ratio was 0.95, indicating that the company had 95% of its short-term liabilities covered by its short-term assets. In 2022, the cash ratio decreased to 0.92, indicating that the company had 92% of its short-term liabilities covered by its short-

In [16]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_cash_ratio'
type_file = '.csv'

cash_ratio_df.to_csv(path+file+type_file, index=True)

# 6. Sentence similarity with Sentence Transformers

Import necessary libraries such as SentenceTransformer, util from sentence_transformers, pandas and torch. Define a function named calculate_cosine_similarity that takes a DataFrame (df) and a Sentence Transformer model (model) as input: Iterate through rows of the DataFrame, calculating cosine similarity between the embeddings of "analysis_expected" and "analysis_generated" columns using the Sentence Transformer model. 
Append the cosine similarity values to a list and add a new column 'cosine_similarity' to the DataFrame. 
Return the modified DataFrame.

In [7]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch

pd.set_option('display.max_colwidth', None)

def calculate_cosine_similarity(df, model):
    cosine_similarities = []
    
    for index, row in df.iterrows():
        expected_embedding = model.encode([row['analysis_expected']])
        generated_embedding = model.encode([row['analysis_generated']])
        cosine_similarity_matrix = util.cos_sim(torch.tensor(expected_embedding), torch.tensor(generated_embedding)).numpy()
        cosine_similarity = cosine_similarity_matrix[0][0]
        cosine_similarities.append(cosine_similarity)

    df['cosine_similarity'] = cosine_similarities

    return df

Load Sentence Transformer Model: Specify the Sentence Transformer model name (st_model_name) as 'sentence-transformers/all-mpnet-base-v2'. Create an instance of the Sentence Transformer model (st_model) using the specified model name.

In [3]:
st_model_name = 'sentence-transformers/all-mpnet-base-v2'
st_model = SentenceTransformer(st_model_name)

## 6.1 Sentence similarity: Current ratio

Load Current ratio DataFrame created in step 5 from CSV file. Perform cosine similarity calculation using the calculate_cosine_similarity function and the Sentence Transformer model. Save the resulting DataFrame with added 'cosine_similarity' column to new CSV file.

In [11]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_current_ratio'
type_file = '.csv'

zs_current_ratio = pd.read_csv(path+file+type_file, index_col = 0)

In [12]:
zs_current_ratio_similarity = calculate_cosine_similarity(zs_current_ratio, st_model)

In [13]:
zs_current_ratio_similarity

,company,ratio,instruction,analysis_expected,analysis_generated,cosine_similarity
0,3M,Current,"Current ratio of a company is 1.70 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.54, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In 2021, the ratio is at ideal values, as the company would be able to meet current liabilities with its current assets and, in addition, short-term assets represent 150% to 200% of the value of short-term liabilities. Thus, the company is also in a position to meet future contingencies. In 2022, the ratio remains at adequate levels. However, its liquidity has seen a slight decrease compared to 2021.","The current ratio of a company is a measure of its ability to pay its short-term liabilities with its short-term assets. A current ratio of 1.70 in 2021 indicates that the company had 1.70 times its current liabilities in current assets. This is considered a healthy ratio, as it shows that the company has enough resources to pay its short-term debts.\n\nIn 2022, the current ratio decreased to 1.54, which indicates that the company had 1.54 times its current liabilities in current assets. This is still considered a healthy ratio, but it shows a slight decrease in the company's ability to pay its short-term debts.\n\nOverall, the current ratio analysis for the years 2021 and 2022 shows that the company's ability to pay its short-term liabilities has remained stable, with a slight decrease in 2022.",0.823912
1,Adobe,Current,"Current ratio of a company is 1.25 in 2021, placing its value in the category defined as between 1.00 and 1.49. In 2022, Current ratio has decreased to 1.11, placing its value in the category defined as between 1.00 and 1.49. What would be the Current ratio analysis for the years 2021 and 2022?","Current ratio, ranging between 1.00 and 1.49 for both 2021 and 2022, indicates that its current assets are likely sufficient to cover short-term debts, but caution is warranted as it may not be well-prepared for unforeseen events affecting liquidity. Although the ratio for both years falls within the same range, it is important to note that the ratio has decreased in 2022 compared to 2021, indicating that liquidity has been reduced.","The current ratio of a company is a measure of its ability to pay its short-term obligations as they come due. A current ratio of 1.25 in 2021 indicates that the company has $1.25 of current assets for every $1 of current liabilities. This suggests that the company has a relatively strong ability to pay its short-term obligations.\n\nIn 2022, the current ratio has decreased to 1.11, indicating that the company now has $1.11 of current assets for every $1 of current liabilities. This suggests that the company's ability to pay its short-term obligations has decreased slightly.\n\nIt is important to note that a current ratio of 1.00 or higher is generally considered to be a healthy sign, indicating that a company has enough current assets to cover its current liabilities. A ratio below 1.00 may indicate that a company is having difficulty paying its short-term obligations.\n\nTherefore, based on the current ratio analysis for the years 2021 and 2022, the company's ability to pay its short-term obligations has decreased slightly, but it is still within a healthy range.",0.831068
2,Airbnb,Current,"Current ratio of a company is 1.95 in 2021, placing its value in the category defined as between 1.50 and 2.00. In 2022, Current ratio has decreased to 1.86, placing its value in the category defined as between 1.50 and 2.00. What would be the Current ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the ratio remains within the ideal range, with short-term assets and receivables surpassing short-term obligations by 50% to 100%. This enables the company to effectively manage unforeseen events im

In [14]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_current_ratio_similarity'
type_file = '.csv'

zs_current_ratio_similarity.to_csv(path+file+type_file, index=True)

## 6.2 Sentence similarity: Quick ratio

Load Quick ratio DataFrame created in step 5 from CSV file. Perform cosine similarity calculation using the calculate_cosine_similarity function and the Sentence Transformer model. Save the resulting DataFrame with added 'cosine_similarity' column to new CSV file.

In [15]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_quick_ratio'
type_file = '.csv'

zs_quick_ratio = pd.read_csv(path+file+type_file, index_col = 0)

In [16]:
zs_quick_ratio_similarity = calculate_cosine_similarity(zs_quick_ratio, st_model)

In [17]:
zs_quick_ratio_similarity

,company,ratio,instruction,analysis_expected,analysis_generated,cosine_similarity
0,3M,Quick,"Quick ratio of a company is 1.08 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 0.93, placing its value in the category defined as less than 1.00. What would be the Quick ratio analysis for the years 2021 and 2022?","It suggests that the company had a relatively healthy liquidity position in 2021. It implies that the company had sufficient liquid assets (such as cash, marketable securities, and accounts receivable, excluding inventory) to cover its short-term liabilities. In 2022 indicates that the liquidity position has weakened compared to the previous year. The quick ratio falling below 1.00 suggests that the company may have difficulty meeting its short-term obligations with its most liquid assets. It implies that the current assets, excluding inventory or other less liquid assets such Prepaid Expenses, may be insufficient to cover the current liabilities. The company might need to rely more on the sale of inventory or other sources to meet its immediate financial obligations.","The quick ratio is a measure of a company's ability to meet its short-term obligations with its most liquid assets. A quick ratio of 1.08 in 2021 indicates that the company has a relatively high level of liquid assets compared to its short-term obligations, and is therefore in a strong position to meet its short-term obligations. A quick ratio of 0.93 in 2022, on the other hand, indicates that the company has a lower level of liquid assets compared to its short-term obligations, and may have a more difficult time meeting its short-term obligations.\n\nOverall, the quick ratio analysis for the years 2021 and 2022 shows a decrease in the company's ability to meet its short-term obligations with its most liquid assets. This suggests that the company may need to take steps to improve its liquidity position, such as increasing its liquid assets or reducing its short-term obligations.",0.852369
1,Adobe,Quick,"Quick ratio of a company is 1.11 in 2021, placing its value in the category defined as between 1.00 and 1.50. In 2022, Quick ratio has decreased to 1.00, placing its value in the category defined as between 1.00 and 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","The company had a healthy liquidity position in 2021. This implies that the company had sufficient liquid assets, such as cash, marketable securities, and accounts receivable (excluding inventory or other less liquid assets such prepaid expenses), to cover its short-term liabilities. In 2022 ratio suggests that the liquidity position has weakened compared to the previous year. While the quick ratio is still between 1.00 and 1.50, the decrease raises concerns about the ability to cover short-term liabilities with the most liquid assets.","The quick ratio is a measure of a company's ability to pay its short-term liabilities with its most liquid assets. It is calculated by dividing the sum of current assets (excluding inventory) by the sum of current liabilities. In 2021, the quick ratio was 1.11, indicating that the company had 1.11 times its current liabilities in current assets (excluding inventory) to cover its short-term debts. In 2022, the quick ratio decreased to 1.00, indicating that the company had only 1.00 times its current liabilities in current assets (excluding inventory) to cover its short-term debts. This suggests that the company's ability to pay its short-term liabilities has decreased.",0.753256
2,Airbnb,Quick,"Quick ratio of a company is 1.89 in 2021, placing its value in the category defined as greater than 1.50. In 2022, Quick ratio has decreased to 1.81, placing its value in the category defined as greater than 1.50. What would be the Quick ratio analysis for the years 2021 and 2022?","In 2021 and 2022, current assets (excluding inventories and other less liquid assets) are more than 

In [18]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_quick_ratio_similarity'
type_file = '.csv'

zs_quick_ratio_similarity.to_csv(path+file+type_file, index=True)

## 6.3 Sentence similarity: Cash ratio

Load Cash ratio DataFrame created in step 5 from CSV file. Perform cosine similarity calculation using the calculate_cosine_similarity function and the Sentence Transformer model. Save the resulting DataFrame with added 'cosine_similarity' column to new CSV file.

In [19]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_cash_ratio'
type_file = '.csv'

zs_cash_ratio = pd.read_csv(path+file+type_file, index_col = 0)

In [20]:
zs_cash_ratio_similarity = calculate_cosine_similarity(zs_cash_ratio, st_model)

In [21]:
zs_cash_ratio_similarity

,company,ratio,instruction,analysis_expected,analysis_generated,cosine_similarity
0,3M,Cash,"Cash ratio of a company is 0.51 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.38, placing its value in the category defined as between 0.20 and 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","The ratio for 2021 surpasses the upper limit, as cash constitutes more than 40% of current liabilities. This implies that the company may have an excess of cash on hand, surpassing what is required to meet its short-term obligations. In this context, there could be opportunities for the company to deploy the surplus cash more efficiently. However, it is worth noting that the ratio value for 2022 decreased and fell in the range between 0.20 and 0.40, coming into alignment with optimal cash levels.","Cash ratio analysis for the years 2021 and 2022 can be as follows:\n\nIn 2021, the cash ratio was 0.51, which is greater than 0.40, indicating that the company had a higher level of liquid assets than the industry average. This suggests that the company had a relatively strong financial position and was in a good position to meet its short-term obligations.\n\nIn 2022, the cash ratio decreased to 0.38, which is between 0.20 and 0.40. This indicates that the company's liquid assets as a percentage of its current liabilities decreased compared to the previous year. This could be a sign of weaker financial position, as the company may have had to rely more on external financing to meet its short-term obligations. However, it is important to consider other factors such as the company's operating cycle and the composition of its current liabilities when interpreting this ratio.",0.835767
1,Adobe,Cash,"Cash ratio of a company is 0.55 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.52, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","Both in 2021 and 2022, cash exceeds appropriate levels, so short-term resources are not being used adequately. Although the ratio has slightly decreased in 2022, the excess of liquidity persists.","The cash ratio analysis for the years 2021 and 2022 would indicate that the company's liquidity position has improved in 2022 compared to 2021. In 2021, the cash ratio was 0.55, which is higher than the value of 0.40 that defines a positive cash ratio. This suggests that the company had a relatively high level of liquidity in 2021. In 2022, the cash ratio decreased to 0.52, which is still greater than 0.40, indicating that the company's liquidity position has improved compared to the previous year. This suggests that the company has been able to generate more cash flow from its operations in 2022, which has helped to increase its liquidity position.",0.687449
2,Airbnb,Cash,"Cash ratio of a company is 0.95 in 2021, placing its value in the category defined as greater than 0.40. In 2022, Cash ratio has decreased to 0.92, placing its value in the category defined as greater than 0.40. What would be the Cash ratio analysis for the years 2021 and 2022?","In both 2021 and 2022, the amount of cash surpasses 40% of short-term current liabilities. This suggests an opportunity to redirect resources for other purposes. It is important to note that, although the ratio has decreased in 2022, the excess liquidity situation has not been resolved.","The cash ratio is a liquidity ratio that measures a company's ability to pay its short-term liabilities with its short-term assets. A cash ratio greater than 1 indicates that a company has enough liquid assets to cover its short-term liabilities. In 2021, the cash ratio was 0.95, indicating that the company had 95% of its short-term liabilities covered by its short-term assets. In 2022, the cash ratio decreased to 0.92, indicating that the company had 92% of its short-te

In [22]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_cash_ratio_similarity'
type_file = '.csv'

zs_cash_ratio_similarity.to_csv(path+file+type_file, index=True)

# 7. Evaluation results

## 7.1 Create a DataFrame template

Define the 'index' and 'columns' for a DataFrame with specific labels for rows and columns. Create an empty DataFrame using the specified index and columns. This dataframe will store the mean cosine similarity for each ratio type.

In [16]:
import pandas as pd

index = ['zero_shot']
columns = ['current_ratio', 'quick_ratio', 'cash_ratio']

df_template = pd.DataFrame(index=index, columns=columns)

df_template

,current_ratio,quick_ratio,cash_ratio
zero_shot,NaN,NaN,NaN


## 7.2 Calculate Cosine Similarity mean

Define a function named 'sentence_similarity_mean' that calculates the mean of the 'cosine_similarity' column in a given DataFrame and formats it with six decimal places. Define a function named 'store_evaluation' that updates the 'df_template' DataFrame with the mean cosine similarity for a specific ratio type using a given DataFrame.

In [17]:
def sentence_similarity_mean(df):
    mean = df['cosine_similarity'].sum() / len(df)
    formatted_mean = '{:.6f}'.format(mean)
    return formatted_mean
    
def store_evaluation(index, ratio_type, df_template, df_results):
    df_template.at[index, ratio_type] = sentence_similarity_mean(df_results)

## Current ratio

Retrieve the file containing cosine similarity results for Current ratio. Set the ratio name to be updated.

In [18]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_current_ratio_similarity'
type_file = '.csv'
ratio_name = 'current_ratio'

zs_current_results = pd.read_csv(path + file + type_file)

Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'current_ratio' column.

In [19]:
store_evaluation(index[0], ratio_name, df_template, zs_current_results)

## Quick ratio

Retrieve the file containing cosine similarity results for Quick ratio. Set the ratio name column to be updated.

In [21]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_quick_ratio_similarity'
type_file = '.csv'
ratio_name = 'quick_ratio'

zs_quick_results = pd.read_csv(path + file + type_file)

Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'quick_ratio' column.

In [22]:
store_evaluation(index[0], ratio_name, df_template, zs_quick_results)

## Cash ratio

Retrieve the file containing cosine similarity results for Cash ratio. Set the ratio name column to be updated.

In [24]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_cash_ratio_similarity'
type_file = '.csv'
ratio_name = 'cash_ratio'

zs_cash_results = pd.read_csv(path + file + type_file)

Call 'store_evaluation' to update the 'df_template' DataFrame with mean cosine similarity values for the 'cash_ratio' column.

In [25]:
store_evaluation(index[0], ratio_name, df_template, zs_cash_results)

## 7.3 Evaluation summary

Display the updated DataFrame template with cosine similarity means and save it to a CSV file.

In [27]:
df_template

,current_ratio,quick_ratio,cash_ratio
zero_shot,0.830240,0.842728,0.831505


In [28]:
path = './Zero_Shot_Evaluation/'
file = 'zero_shot_similarity_summary'
type_file = '.csv'

df_template.to_csv(path+file+type_file, index=True)